In [ ]:
import requests
import pandas as pd
import numpy as np
import json

cpiUrl = "https://data.api.abs.gov.au/rest/data/ABS,CPI,1.1.0/3.10001.10.50.Q?lastNObservations=20&detail=dataonly"
wpiUrl = "https://data.api.abs.gov.au/rest/data/ABS,WPI,1.2.0/3.THRPIB.2.TOT.10.AUS.Q?lastNObservations=20&detail=dataonly"
rentUrl = "https://data.api.abs.gov.au/rest/data/ABS,CPI,1.1.0/3.30014.10.50.Q?lastNObservations=20&detail=dataonly"
saleUrl = "https://Url.api.abs.gov.au/rest/data/ABS,AUSTRALIAN_INDUSTRY,1.1.0/INCSALGDSSERV..1..A?lastNObservations=5&detail=dataonly"
cogsUrl = "https://Url.api.abs.gov.au/rest/data/ABS,AUSTRALIAN_INDUSTRY,1.1.0/PURCHASES..1..A?lastNObservations=5&detail=dataonly"

headers = {
    "Accept": "application/vnd.sdmx.data+json"
}

def fetch_raw_json(name, url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return
        print(f"Failed to fetch {name}: {response.status_code} - {response.text}")

def AusIndustryUrl (measure, industry='', region=''):
    return f"https://data.api.abs.gov.au/rest/data/ABS,AUSTRALIAN_INDUSTRY,1.1.0/{measure}.{industry}.1.{region}.A?lastNObservations=5&detail=dataonly"

capitalsKeys = {
    "Sydney": "1",
    "Melbourne": "2",
    "Brisbane": "3",
    "Adelaide": "4",
    "Perth": "5",
    "Hobart": "6",
    "Darwin": "7",
    "Canberra": "8"}

regionKeys = {
    "New South Wales": "1",
    "Victoria": "2",
    "Queensland": "3",
    "South Australia": "4",
    "Western Australia": "5",
    "Tasmania": "6",
    "Northern Territory": "7",
    "Australian Capital Territory": "8",
}
industryKeys = {
    "Wholesale Trade": "F",
    "Retail Trade": "G",
    "Accommodation and Food Services": "H",
    "Transport, Postal and Warehousing": "I",
    "Information Media and Telecommunications": "J",
    "Rental, Hiring and Real Estate Services": "L",
    "Administrative and Support Services": "N",
    "Education and Training": "P",
    "Health Care and Social Assistance": "Q",
    "Arts and Recreation Services": "R",
    "Other Services": "S",    
    }
divisionKeys = {
    'Motor vehicle and motor vehicle parts retailing':'39',
    'Fuel retailing':'40',
    'Food retailing':'41',
    'Other store-based retailing':'42',
    'Non-store retailing and retail commission-based buying and/or selling':'43',
    'Accommodation':'44',
    'Food and beverage services':'45',
    'Rental and hiring services (except real estate)': '66',
    'Real estate services': '67',
    'Nail, Hair, and Skin care service':'95',
}

cpiData = fetch_raw_json("CPI Data", cpiUrl)
cpiData = cpiData["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
cpiValues = [float(v[0]) for v in cpiData.values()]
cpiAvg = sum(cpiValues) / len(cpiValues)
cpiAvg

wpiData = fetch_raw_json("WPI Data", wpiUrl)
wpiData = wpiData["data"]["dataSets"][0]["series"]["0:0:0:0:0:0:0"]["observations"]
wpiValues = [float(v[0]) for v in wpiData.values()]
wpiAvg = sum(wpiValues) / len(wpiValues)
wpiAvg

rentalYields = {}
for key in capitalsKeys:
    try:
        rentalData = fetch_raw_json(f"Rental Data for {key}", f"https://data.api.abs.gov.au/rest/data/ABS,CPI,1.1.0/3.30014.10.{capitalsKeys[key]}.Q?lastNObservations=20&detail=dataonly")["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
        rentalIndex = [float(v[0]) for v in rentalData.values()]
        avgRentalYield = round(sum(rentalIndex) / len(rentalIndex), 2)
        rentalYields[key] = avgRentalYield
    except Exception as e:
        rentalYields[key] = None

dataByState = {}
for key in regionKeys:
    try:
        data = fetch_raw_json(f"Sales Data for {key}", AusIndustryUrl("INCSALGDSSERV", region = regionKeys[key]))["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
        sale = [float(v[0]) for v in data.values()]
        dataByState[key] = {"Sales": sale,
                            "CAGR": round(((sale[0] / sale[4]) ** (1/5) - 1) * 100, 2)}
    except Exception as e:
        dataByState[key] = None

dataByIndustry = {}
for key in industryKeys:
    try:
        saleData = fetch_raw_json(f"Sales Data for {key}", AusIndustryUrl("INCSALGDSSERV", industry = industryKeys[key], region = 'AUS'))["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
        cogsData = fetch_raw_json(f"Purchases Data for {key}", AusIndustryUrl("PURCHASES", industry = industryKeys[key], region = 'AUS'))["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
        sale = [float(v[0]) for v in saleData.values()]
        saleCAGR = round(((sale[0] / sale[4]) ** (1/5) - 1) * 100, 2)

        cogs = [float(v[0]) for v in cogsData.values()]
        stockLevel =  [cogs[i]  / sale[i] * 100 for i in range(len(sale))]
        avgStockLevel = round(sum(stockLevel)/len(stockLevel),2)
        dataByIndustry[key] = {
            "Sales": sale,
            "CAGR": saleCAGR,
            "Purchases": cogs,
            "stockLevel": avgStockLevel,
        }
    except Exception as e:
        dataByIndustry[key] = None

marketData = {
    "CPI": cpiAvg,
    "WPI": wpiAvg,
    "growthByState": [{r : dataByState[r]["CAGR"]} for r in regionKeys],
    "growthByIndustry": [{i : dataByIndustry[i]["CAGR"]} for i in industryKeys], 
    "stockLevelByIndustry": [{i : dataByIndustry[i]["stockLevel"]} for i in industryKeys],
    "rentalYieldByCapital": rentalYields,
},

marketData

Data obtained for Wholesale Trade 77.85
Data obtained for Retail Trade 70.73
Data obtained for Accommodation and Food Services 33.11
Data obtained for Transport, Postal and Warehousing 16.07
Data obtained for Information Media and Telecommunications 19.36
Data obtained for Rental, Hiring and Real Estate Services 12.71
Data obtained for Administrative and Support Services 7.55
Data obtained for Education and Training 7.57
Data obtained for Health Care and Social Assistance 10.23
Data obtained for Arts and Recreation Services 8.69
Data obtained for Other Services 31.02


({'CPI': 3.825,
  'WPI': 2.645,
  'growthByState': [{'New South Wales': 6.83},
   {'Victoria': 2.19},
   {'Queensland': 9.34},
   {'South Australia': 6.22},
   {'Western Australia': 7.8},
   {'Tasmania': 3.95},
   {'Northern Territory': 7.28},
   {'Australian Capital Territory': 4.96}],
  'growthByIndustry': [{'Wholesale Trade': 6.87},
   {'Retail Trade': 6.5},
   {'Accommodation and Food Services': 8.56},
   {'Transport, Postal and Warehousing': 6.35},
   {'Information Media and Telecommunications': 5.22},
   {'Rental, Hiring and Real Estate Services': 5.49},
   {'Administrative and Support Services': 7.15},
   {'Education and Training': 14.69},
   {'Health Care and Social Assistance': 8.94},
   {'Arts and Recreation Services': 7.76},
   {'Other Services': 7.54}],
  'stockLevelByIndustry': [{'Wholesale Trade': 77.85},
   {'Retail Trade': 70.73},
   {'Accommodation and Food Services': 33.11},
   {'Transport, Postal and Warehousing': 16.07},
   {'Information Media and Telecommunications

In [ ]:
import json
import boto3

def fetch_raw_json(name, url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return
        print(f"Failed to fetch {name}: {response.status_code} - {response.text}")

def AusIndustryUrl (measure, industry='', region=''):
    return f"https://data.api.abs.gov.au/rest/data/ABS,AUSTRALIAN_INDUSTRY,1.1.0/{measure}.{industry}.1.{region}.A?lastNObservations=5&detail=dataonly"

def lambda_handler(event, context):
    # Your code goes here, wrapped in this function
    capitalsKeys = {
    "Sydney": "1",
    "Melbourne": "2",
    "Brisbane": "3",
    "Adelaide": "4",
    "Perth": "5",
    "Hobart": "6",
    "Darwin": "7",
    "Canberra": "8"}

    regionKeys = {
    "New South Wales": "1",
    "Victoria": "2",
    "Queensland": "3",
    "South Australia": "4",
    "Western Australia": "5",
    "Tasmania": "6",
    "Northern Territory": "7",
    "Australian Capital Territory": "8",
    }
    industryKeys = {
    "Wholesale Trade": "F",
    "Retail Trade": "G",
    "Accommodation and Food Services": "H",
    "Transport, Postal and Warehousing": "I",
    "Information Media and Telecommunications": "J",
    "Rental, Hiring and Real Estate Services": "L",
    "Administrative and Support Services": "N",
    "Education and Training": "P",
    "Health Care and Social Assistance": "Q",
    "Arts and Recreation Services": "R",
    "Other Services": "S",    
    }
    divisionKeys = {
    'Motor vehicle and motor vehicle parts retailing':'39',
    'Fuel retailing':'40',
    'Food retailing':'41',
    'Other store-based retailing':'42',
    'Non-store retailing and retail commission-based buying and/or selling':'43',
    'Accommodation':'44',
    'Food and beverage services':'45',
    'Rental and hiring services (except real estate)': '66',
    'Real estate services': '67',
    'Nail, Hair, and Skin care service':'95',
}

    cpiData = fetch_raw_json("CPI Data", cpiUrl)
    cpiData = cpiData["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
    cpiValues = [float(v[0]) for v in cpiData.values()]
    cpiAvg = sum(cpiValues) / len(cpiValues)
    cpiAvg

    wpiData = fetch_raw_json("WPI Data", wpiUrl)
    wpiData = wpiData["data"]["dataSets"][0]["series"]["0:0:0:0:0:0:0"]["observations"]
    wpiValues = [float(v[0]) for v in wpiData.values()]
    wpiAvg = sum(wpiValues) / len(wpiValues)
    wpiAvg

    rentalYields = {}
    for key in capitalsKeys:
        try:
            rentalData = fetch_raw_json(f"Rental Data for {key}", f"https://data.api.abs.gov.au/rest/data/ABS,CPI,1.1.0/3.30014.10.{capitalsKeys[key]}.Q?lastNObservations=20&detail=dataonly")["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
            rentalIndex = [float(v[0]) for v in rentalData.values()]
            avgRentalYield = round(sum(rentalIndex) / len(rentalIndex), 2)
            rentalYields[key] = avgRentalYield
        except Exception as e:
            rentalYields[key] = None

    dataByState = {}
    for key in regionKeys:
        try:
            data = fetch_raw_json(f"Sales Data for {key}", AusIndustryUrl("INCSALGDSSERV", region = regionKeys[key]))["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
            sale = [float(v[0]) for v in data.values()]
            dataByState[key] = {"Sales": sale,
                            "CAGR": round(((sale[0] / sale[4]) ** (1/5) - 1) * 100, 2)}
        except Exception as e:
            dataByState[key] = None

    dataByIndustry = {}
    for key in industryKeys:
        try:
            saleData = fetch_raw_json(f"Sales Data for {key}", AusIndustryUrl("INCSALGDSSERV", industry = industryKeys[key], region = 'AUS'))["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
            cogsData = fetch_raw_json(f"Purchases Data for {key}", AusIndustryUrl("PURCHASES", industry = industryKeys[key], region = 'AUS'))["data"]["dataSets"][0]["series"]["0:0:0:0:0"]["observations"]
            sale = [float(v[0]) for v in saleData.values()]
            saleCAGR = round(((sale[0] / sale[4]) ** (1/5) - 1) * 100, 2)

            cogs = [float(v[0]) for v in cogsData.values()]
            stockLevel =  [cogs[i]  / sale[i] * 100 for i in range(len(sale))]
            avgStockLevel = round(sum(stockLevel)/len(stockLevel),2)
            dataByIndustry[key] = {
            "Sales": sale,
            "CAGR": saleCAGR,
            "Purchases": cogs,
            "stockLevel": avgStockLevel,
        }
        except Exception as e:
            dataByIndustry[key] = None

    marketData = {
        "CPI": cpiAvg,
    "WPI": wpiAvg,
    "growthByState": [{r : dataByState[r]["CAGR"]} for r in regionKeys],
    "growthByIndustry": [{i : dataByIndustry[i]["CAGR"]} for i in industryKeys], 
    "stockLevelByIndustry": [{i : dataByIndustry[i]["stockLevel"]} for i in industryKeys],
    "rentalYieldByCapital": rentalYields,
    },

    # Replace the last line with:
    s3 = boto3.client('s3')
    s3.put_object(
        Bucket='your-s3-bucket-name',
        Key='data/marketData.json',  # or any path inside the bucket
        Body=json.dumps(marketData, indent=2),
        ContentType='application/json'
    )

    return {
        'statusCode': 200,
        'body': json.dumps("Data updated successfully")
    }